In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.keras import applications
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Flatten, Dense, Dropout

/Users/karenliu/miniconda2/envs/ai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_weights.h5'
top_model_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/training'
validation_data_dir = 'data/validation'
nb_train_samples = 480
nb_validation_samples = 120
epochs = 5
batch_size = 4
class_indices = None

def save_bottlebeck_features():
    global class_indices
    datagen = ImageDataGenerator(
        rescale=1. / 255,
        horizontal_flip=True,
        width_shift_range=0.10,
        rotation_range=20)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
    
    if not class_indices:
        class_indices = generator.class_indices
    print('Generator class_indices: ', generator.class_indices)
    #bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    #np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
    target_batches = nb_train_samples // batch_size
    X_validation = np.zeros(shape=(target_batches*batch_size, 4, 4, 512))
    Y_validation = np.zeros(shape=(target_batches*batch_size, generator.num_class))
    batches = 0
    for x_batch, y_batch in generator:
        bottleneck_features = model.predict_on_batch(x_batch)
        #print('bottleneck_features shape: ', bottleneck_features.shape)
        X_validation[batches*batch_size:(batches+1)*batch_size] = bottleneck_features
        Y_validation[batches*batch_size:(batches+1)*batch_size] = y_batch
        batches += 1
        if batches >= target_batches:
            break

    print('Training Bottleneck: ', np.shape(X_validation), np.shape(Y_validation))
    np.savez(open('bottleneck_features_train.npy', 'wb'), X=X_validation, Y=Y_validation)

    ## Generating Validation Bottleneck features
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
    #bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size)
    #np.savez(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)
    target_batches = nb_validation_samples // batch_size
    X_validation = np.zeros(shape=(target_batches*batch_size, 4, 4, 512))
    Y_validation = np.zeros(shape=(target_batches*batch_size, generator.num_class))
    batches = 0
    for x_batch, y_batch in generator:
        bottleneck_validation = model.predict_on_batch(x_batch)
        #print('bottleneck_validation shape: ', bottleneck_validation.shape)
        X_validation[batches*batch_size:(batches+1)*batch_size] = bottleneck_validation
        Y_validation[batches*batch_size:(batches+1)*batch_size] = y_batch
        batches += 1
        if batches >= target_batches:
            break

    print('Validation Bottleneck: ', np.shape(X_validation), np.shape(Y_validation))
    np.savez(open('bottleneck_features_validation.npy', 'wb'), X=X_validation, Y=Y_validation)

In [3]:
save_bottlebeck_features()

Found 48 images belonging to 3 classes.
Generator class_indices:  {'JapaneseBobtail': 0, 'Mainecoon': 1, 'Ragdoll': 2}
Training Bottleneck:  (480, 4, 4, 512) (480, 3)
Found 12 images belonging to 3 classes.
Validation Bottleneck:  (120, 4, 4, 512) (120, 3)


In [4]:
def load_bottleneck_features():
    features = np.load(open('bottleneck_features_validation.npy', 'rb'))
    print(features['X'].shape)
    print(features['Y'].shape)
    
load_bottleneck_features()

(120, 4, 4, 512)
(120, 3)


In [5]:
model_classes = 3
def train_top_model():
    train_arrays = np.load(open('bottleneck_features_train.npy', 'rb'))
    #train_labels = np.array([0] * (int(nb_train_samples / 2)) + [1] * (int(nb_train_samples / 2)))
    train_data = train_arrays['X']
    train_labels = train_arrays['Y']
    
    validation_arrays = np.load(open('bottleneck_features_validation.npy', 'rb'))
    #validation_labels = np.array([0] * (int(nb_validation_samples / 2)) + [1] * (int(nb_validation_samples / 2)))
    validation_data = validation_arrays['X']
    validation_labels = validation_arrays['Y']

    model = Sequential() # tf.keras.models.Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(model_classes, activation='softmax'))

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    model.save(top_model_path)


In [6]:
epochs = 10
train_top_model()

Train on 480 samples, validate on 120 samples
Epoch 1/10
480/480 [==============================] - 4s - loss: 0.7516 - acc: 0.8479 - val_loss: 0.1527 - val_acc: 0.9417
Epoch 2/10
480/480 [==============================] - 4s - loss: 0.0748 - acc: 0.9729 - val_loss: 0.0490 - val_acc: 0.9917
Epoch 3/10
480/480 [==============================] - 3s - loss: 0.0589 - acc: 0.9813 - val_loss: 0.0669 - val_acc: 0.9583
Epoch 4/10
480/480 [==============================] - 3s - loss: 0.0573 - acc: 0.9854 - val_loss: 0.0757 - val_acc: 0.9667
Epoch 5/10
480/480 [==============================] - 3s - loss: 0.0587 - acc: 0.9896 - val_loss: 0.2863 - val_acc: 0.9000
Epoch 6/10
480/480 [==============================] - 3s - loss: 0.0081 - acc: 0.9958 - val_loss: 0.4459 - val_acc: 0.9167
Epoch 7/10
480/480 [==============================] - 3s - loss: 0.0041 - acc: 0.9979 - val_loss: 0.2070 - val_acc: 0.9333
Epoch 8/10
480/480 [==============================] - 3s - loss: 9.6884e-04 - acc: 1.0000 - v

In [7]:
from tensorflow.python.keras.models import load_model
from PIL import Image as pil_image

# build the VGG16 network
vgg_model = applications.VGG16(include_top=False, weights='imagenet')
def calculate_bottleneck_for_testing(img_path, model, rescale=1. / 255):
    image_shape = (img_width, img_height, 3)
    img = pil_image.open(img_path)
    img = img.resize((img_width, img_height))
    x = np.asarray(img, dtype=float)
    x = x * rescale
    X = np.zeros(shape=(1, img_width, img_height, 3))
    X[0] = x
    bottleneck_features = model.predict_on_batch(X)
    return bottleneck_features

loaded_model = load_model(top_model_path)
def predict_breed(cat_img, kera_model):
    bottleneck_test = calculate_bottleneck_for_testing(cat_img, vgg_model)
    X_test = np.zeros(shape=(1, 4, 4, 512))
    X_test[0] = bottleneck_test
    return kera_model.predict(X_test)


### Testing Out Classifier With This Cat Image:
<img src="data/test/cat2.jpg",width=150,height=150>

In [8]:
print('class_indices: ', class_indices)

#result = predict_breed('data/test/Ragdoll/ragdoll0001.jpg', loaded_model)
#print(result)

#print(predict_breed('data/training/Ragdoll/ragdoll0016.jpg', loaded_model))
print(predict_breed('data/test/cat2.jpg', loaded_model))

print(predict_breed('data/test/Cat.jpg', loaded_model))


class_indices:  {'JapaneseBobtail': 0, 'Mainecoon': 1, 'Ragdoll': 2}
[[  6.37534662e-14   1.55650205e-13   1.00000000e+00]]
[[  3.12105112e-04   8.99790964e-10   9.99687910e-01]]
